In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pickle

### 1. Loading the data



In [ ]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

### 2. Computing duration

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [ ]:
sns.distplot(df.duration)

In [ ]:
df.duration.describe(percentiles=[.95, .98, .99])

### 3. Dropping outliers

In [ ]:
len(df[(df.duration >= 1) & (df.duration <= 60)]) / len(df) * 100

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [ ]:
df.shape

In [ ]:
sns.histplot(df.duration)

### 4. One-hot encoding

In [ ]:
df.head()

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

In [ ]:
dv = DictVectorizer()

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train

In [ ]:
print(f'Feature matrix size: {X_train.shape}')

### 5. Training a model

In [ ]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression() # Ridge(0.8) # # Lasso(0.1) #, Ridge
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f'Train RMSE: {mean_squared_error(y_train, y_pred, squared=False)}')

In [ ]:
sns.histplot(y_train, label='actual')
sns.histplot(y_pred, label='prediction')

plt.legend();

### 6. Evaluating the model

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [ ]:
df_val = read_data('../data/yellow_tripdata_2023-02.parquet')

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [ ]:
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
print(f'Val RMSE: {mean_squared_error(y_val, y_pred, squared=False)}')

In [ ]:
sns.histplot(y_val, label='actual')
sns.histplot(y_pred, label='prediction')

plt.legend();

### 7. Improve score

In [ ]:
df_train = read_data('../data/yellow_tripdata_2023-01.parquet')
df_val = read_data('../data/yellow_tripdata_2023-02.parquet')

In [ ]:
df_train

#### 7.1 Compute features

In [ ]:
# use ['PULocationID', 'DOLocationID'] as combined feature

In [ ]:
df_train['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val['PU_DO'] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [ ]:
df_train.head()

#### 7.2 Launch training

In [ ]:
categorical = ['PU_DO']
target = 'duration'

In [ ]:
dv = DictVectorizer()
train_dicts = df_train[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
y_train = df_train[target].values


lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
print(f'Train RMSE: {mean_squared_error(y_train, y_train_pred, squared=False)}')

In [ ]:
sns.histplot(y_train, label='actual')
sns.histplot(y_train_pred, label='prediction')

plt.legend()

#### 7.3 Run Validation

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts) 
y_val = df_val[target].values

y_val_pred = lr.predict(X_val)
print(f'Train RMSE: {mean_squared_error(y_val, y_val_pred, squared=False)}')

In [ ]:
sns.distplot(y_val, label='actual')
sns.distplot(y_val_pred, label='prediction')

plt.legend()

### 8. Save model

In [ ]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
!ls -al ../models

### 9. MLFlow

In [ ]:
import mlflow

In [ ]:
mlflow.set_tracking_uri(uri="sqlite:///../mlruns.db")

In [ ]:
mlflow.set_experiment("DU-AI")

In [ ]:
def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    
    return df

In [ ]:
train_data_path = '../data/yellow_tripdata_2023-01.parquet'
val_data_path = '../data/yellow_tripdata_2023-02.parquet'

In [ ]:
df_train = read_data(train_data_path)
df_val = read_data(val_data_path)

In [ ]:
# Prepare data
categorical = ['PULocationID', 'DOLocationID']

# cast types to string
df_train[categorical] = df_train[categorical].astype('str')
df_val[categorical] = df_val[categorical].astype('str')

# get columns as dict
train_dicts = df_train[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')

# fit transform dict vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y_train = df_train.duration.values
y_val = df_val.duration.values

In [ ]:
y_train.shape, y_val.shape

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("dev", "naji")
    mlflow.set_tag("change", "preprocessing")
    mlflow.log_param("train dp", train_data_path)
    mlflow.log_param("val dp", val_data_path)

    # alpha = 0.5
    lr = LinearRegression() # Ridge(alpha)
    mlflow.log_param("model type", type(lr))
    # mlflow.log_param("alpha", alpha)
    
    lr.fit(X_train, y_train)
    
    y_train_pred = lr.predict(X_train)
    train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
    mlflow.log_metric("Train RMSE", train_rmse)
    
    y_val_pred = lr.predict(X_val)
    val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    mlflow.log_metric("Val RMSE", val_rmse)

In [ ]:
print(f'Train RMSE: {train_rmse}')
print(f'Val RMSE: {val_rmse}')